In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
import backtest_tools.portfolio_tools as pt

In [3]:
asset_cols = pd.read_csv(r'C:\Users\marcu\Documents\Profession\Windermere Capital\Macro Model\Asset Returns\macro_model_asset_log_returns_filtered.csv').iloc[:,1:].columns

# Get Daily Log & Raw Returns:

In [4]:
import yfinance as yf
assets = asset_cols
log_asset_returns = pd.DataFrame()
raw_asset_returns = pd.DataFrame()

i = 0
for ticker in assets:
    prices = yf.download(ticker, start='1997-01-01', end=dt.date.today())['Adj Close']
    
    # Shift yesterday's prices forward to compute log returns
    log_rets = np.log(prices/prices.shift(1)).dropna()
    log_rets = log_rets.rename(ticker)
    log_asset_returns = pd.concat([log_asset_returns, log_rets], axis=1)

    # Get raw asset returns
    raw_rets = prices.pct_change().dropna()
    raw_rets = raw_rets.rename(ticker)
    raw_asset_returns = pd.concat([raw_asset_returns, raw_rets], axis=1)

    i += 1
    if i > 5:
        break


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [60]:
import backtest_tools.portfolio_tools as pt

perf = pd.DataFrame()
raw_asset_returns = raw_asset_returns.dropna()
for i in raw_asset_returns.columns:
    try:
        perf = pd.concat([perf, pd.Series(pt.performance_summary(raw_asset_returns[i]), name=i)], axis=1)
        print(i)
    except:
        print(f'skipped{i}')
    
perf

AAPL
AMZN
EWA
EWG
EWQ
EWL


,AAPL,AMZN,EWA,EWG,EWQ,EWL
Sharpe Ratio,0.855680,0.746255,0.388643,0.299406,0.370599,0.409123
Drift Adjusted Sharpe Ratio,0.035168,0.041120,0.202353,0.113980,0.108078,0.156832
CAGR,0.314310,0.311742,0.071409,0.045039,0.063601,0.066974
Vol,0.428077,0.573532,0.273035,0.264879,0.256965,0.216030
Sortino Ratio,1.160304,1.101851,0.509976,0.402848,0.491989,0.548481
Martin (Ulcer) Ratio,0.000708,0.000660,0.000397,0.000187,0.000272,0.000361
Omega Ratio,1.140023,1.134478,1.026646,1.008045,1.019832,1.017525
RR Ratio,1.174425,1.161752,1.075289,1.056089,1.070175,1.076947
Win Rate,0.523013,0.510214,0.528157,0.528260,0.529784,0.525963
Skew,-0.566197,1.047367,-0.042274,0.042775,-0.169090,-0.185062


In [17]:
def get_log_returns(prices: pd.Series) -> pd.Series:
    """ Get log returns from a price series.

    Args:
        prices (pd.Series): price time series of a given secrity.

    Return:
        return (pd.Series): time series of log returns of a given security.
    """

    log_rets = np.log(prices/prices.shift(1)).dropna()

    return log_rets

def get_raw_returns(prices: pd.Series) -> pd.Series:
    """ Get raw (non-log) returns from a price series.

    Args:
        prices (pd.Series): price time series of a given security.

    Return:
        return (pd.Series): time series of raw (non-log) returns of a given security.
    """
    
    raw_rets = prices.pct_change().dropna()

    return raw_rets

# Download Intrday Data

In [43]:
from datetime import date

tickers = assets

log_intraday_asset_returns = pd.DataFrame()
raw_intraday_asset_returns = pd.DataFrame()

for ticker in tickers:

    intraday_prices = yf.download(ticker, start=date.today() - pd.DateOffset(58), end=date.today(), interval='5m')['Adj Close'].dropna()
    
    # Shift yesterday's prices forward to compute log returns
    log_rets = get_log_returns(intraday_prices)
    log_rets = log_rets.rename(ticker)
    log_intraday_asset_returns = pd.concat([log_intraday_asset_returns, log_rets], axis=1)

    # Get raw asset returns
    raw_rets = get_raw_returns(intraday_prices)
    raw_rets = raw_rets.rename(ticker)
    raw_intraday_asset_returns = pd.concat([raw_intraday_asset_returns, raw_rets], axis=1)